imports

In [ ]:
# imports 
import pandas as pd
import numpy as np
import json

loading data

In [ ]:
# location dataframe
location_df = pd.read_csv('SaO_Optilandia_resub_locations.csv')

# links dataframe
links_df = pd.read_csv('SaO_Optilandia_resub_links.csv')

# extract lorry data from json
lorry_data = json.load(open('SaO_Optilandia_resub_depot_lorries.json', 'r'))

# set count to 0
k = 0

# initialise lorry list
lorry = []

# loop -> set i to the respective lorry key
for i in lorry_data.keys():
    # set j to the the number of lorries at key 'i'
    for j in range(0, len(lorry_data[i])):
        # append each lorry in lorry_data to lorry list
        lorry.append(pd.DataFrame(lorry_data[i][j], index=[k]))
        # accumulate index
        k += 1

# lorry dataframe
lorry_df = pd.concat(lorry)

In [ ]:
# setting required column 
location_df['required'] = location_df['capacity']-location_df['level']

# displaying rows where is_customer true
location_df[location_df['is_customer']==True].head()

In [ ]:
# add 'depot' column to lorry_df and updae with the lorry's relative depot 
lorry_df['depot'] = lorry_df.lorry_id.apply(lambda x: x.split('-')[0])
lorry_df.head()

In [ ]:
# list of depot locations (where nodes == depot)
depot_locations = np.where(location_df.is_depot)[0]

# list of customer locations (where nodes == customers)
customer_locations = np.where(location_df.is_customer)[0]

imports + visualising the map

In [ ]:
# imports
import networkx as nx
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt 

# parwise distance calculation for each node
euclidean = squareform(pdist(location_df[['x', 'y']]))

# edges list initialisation
edges = []

# loop through links_df rows
for _, (i, j) in links_df.iterrows():
    # append node at i, node at j, and their pairwise distance to edges
    edges.append((i, j, euclidean[i, j]))

# pos dict intialisation
pos = {}

# loop through location_df rows
for k, v in location_df[['x', 'y']].iterrows():
    # update pos dict with array of k, v 
    pos.update({k:v.values})

# initialise depot_labels dict
depot_labels = {}

# loop throgugh depot_locations
for i in depot_locations:
    # update depot_labels dict with {i:i}
    depot_labels.update({i:i})

# initialise customer_labels dict
customer_labels = {}

# loop through customer_locations
for i in customer_locations:
    # update customer_labels dict with {i:i}
    customer_labels.update({i:i})

# initialise nx Graph
G = nx.Graph()

# feed node list to G
G.add_nodes_from(location_df['id'].to_numpy())

# feed edges list to G
G.add_weighted_edges_from(edges)

# resize figure 
plt.figure(figsize=(16, 8))

# sketch graph
nx.draw(G, pos=pos, node_size=40)

# label depot nodes
nx.draw_networkx_labels(G, pos, depot_labels)

# label customer nodes
nx.draw_networkx_labels(G, pos, customer_labels)

# mark depot nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=depot_locations, node_color='r', node_size=400, alpha=0.9)

# mark customer nodes
nx.draw_networkx_nodes(G, pos=pos, nodelist=customer_locations, node_color='g', node_size=200, alpha=0.3)

clustering nodes ... (nearest-neighbour approach)

In [ ]:
# intialise cluster dict
cluster = {124:[], 127:[], 167:[], 523:[]}

# intialise nodes list
nodes = [] 

# loop through each node in customer_locations
for node in customer_locations:
    # check if node in nodes
    if node not in nodes:
        # initialise dist list
        dist = []
        # loop through each depot key
        for depot in cluster.keys():
            # append euclidean weights to dist 
            dist.append(euclidean[node, depot])
        # get shortest distance
        shortestDist = min(dist)
        # match shortest distance to equivalent node index
        nearestDepotIndex = np.where(euclidean[node]==shortestDist)
        # add node to relative nearest depot location
        cluster[int(nearestDepotIndex[0])].append(node)
        # track applied nodes
        nodes.append(node)
        # clear dist
        dist.clear()

# print allocated nodes to relative cluster points (depot locations)
print(cluster)

# clear nodes list
nodes.clear()

(breadth-first-search used for pathfinding)

In [ ]:
# Routing from A to B using recursive Breadth-First-Search based algorithm (pathfinding)
def routing(currState, toState, edges):
    # intialise visitedState list for tracking node traversal
    visitedState = [currState]
    # initialise visitedEdge list for tracking edge traversal
    visitedEdge = []
    # intialise edgeMemory list for storing the explored edges
    edgeMemory = []
    # intialise queue list for choosing the central node for next traversal
    queue = [currState]

    # loop while toState is not found
    while currState != toState:
        # remove and store the last element of the queue list as q
        q = queue.pop(0)
        
        # intialise currEdges dict which holds the next set of edges for traversals
        currEdges = {}
        # get the nodes at each edge, where either nodes are equivalent to q
        for edge in list(np.where(links_df[['id1', 'id2']]==q)[0]):
            # update the dict with the relative edge key and the node pairs
            currEdges.update({edge:[edges[edge][0], edges[edge][1]]})
        
        # store the explored edges in edgeMemory list
        edgeMemory.append(currEdges)

        # loop through each edge in currEdges
        for edge in currEdges:
            # check if the edge has been visited 
            if edge not in visitedEdge:
                # if not visited then add the edge to visitedEdge
                visitedEdge.append(edge)
                # check the node index in the edge that has not been visited 
                if currEdges[edge][0] not in visitedState and currEdges[edge][1] in visitedState:
                    # set currState to the unvisited node 
                    currState = currEdges[edge][0]
                    # mark the node in currState as visited 
                    visitedState.append(currState)
                    # add new currState to queue 
                    queue.append(currState)
                    # check if toState reached
                    if currState == toState:
                        # set currState to toState
                        currState = toState
                        # end loop
                        break 
                # similar to above but in the context of different index position of the node that has not been visited
                if currEdges[edge][1] not in visitedState and edges[edge][0] in visitedState:
                    currState = currEdges[edge][1]
                    visitedState.append(currState)
                    queue.append(currState)
                    if currState == toState:
                        currState = toState
                        break
    
    # set startState as the first node in visitedState list
    startState = visitedState[0]
    # set lastQ as the toState for tracking q node from end of order
    lastQ = [toState]
    # intialise backtrace list for backtracing the edges from edgeMemory
    backtrace = []
    # initialise nodetrace list for backtracing the nodes from edgeMemory
    nodetrace = []

    # intialise edgeMemoryReversed for reordering edgeMemory 
    edgeMemoryReversed = []
    # loop through each index between range 0 and length of edgeMemory
    for i in range (0, len(edgeMemory)):
        # set endElement to the last element in edgeMemory
        endElement = edgeMemory.pop(-1)
        # add the endElement to edgeMemoryReversed
        edgeMemoryReversed.append(endElement)
    
    # while last element in lastQ is not equivalent to the startState
    while lastQ[-1] != startState:
        # loop through each edge options in edgeMemoryReversed
        for edgeOpt in edgeMemoryReversed:
            # loop through each edge from as keys of the edge options
            for edge in edgeOpt.keys():

                # check if last element of lastQ is in the set of edge options given the edge
                if lastQ[-1] in edgeOpt[edge]:
                    # add the edge to backtrace 
                    backtrace.append(edge)
                    # check index of node which matches the lastQ element 
                    if lastQ[-1] == edgeOpt[edge][0] and lastQ[-1] != edgeOpt[edge][1]:
                        # update lastQ as the the node which does not match the lastQ element
                        lastQ.append(edgeOpt[edge][1])
                        # add the node to nodetrace
                        nodetrace.append(edgeOpt[edge][1])
                        # return to while iterate
                        break
                        # similar to above but in the context of different index postion of the matching node with lastQ element
                    if lastQ[-1] != edgeOpt[edge][0] and lastQ[-1] == edgeOpt[edge][1]:
                        lastQ.append(edgeOpt[edge][0])
                        nodetrace.append(edgeOpt[edge][0])
                        break
    
    # re-ordering edges from start to end
    edgeTraversed = []
    for i in range(0, len(backtrace)):
        endElement = backtrace.pop(-1)
        edgeTraversed.append(endElement)

    # re-ordering nodes from start to end 
    nodeOrder = []
    for i in range(0, len(nodetrace)):
        endElement = nodetrace.pop(-1)
        nodeOrder.append(endElement)
    
    # adding route weight (distance between nodes) to each traversal made
    routeWeight = []
    for edge in edgeTraversed:
        routeWeight.append(edges[edge][2])

    # return the the order in which nodes were visited and the order in which edges were traversed
    return nodeOrder, edgeTraversed, routeWeight

In [ ]:
# test: gives the node order in routing
print(f'node order: {routing(124, 10, edges)[0]}')
# test: gives the traversed edges in routing
print(f'edge order: {routing(124, 10, edges)[1]}')
# test: gives the weight routes for traversed
print(f'weights: {routing(124, 10, edges)[2]}')

In [ ]:
# Assign next element in list as customer (no need for distance calc. here since it is considered in Genetic Algorithm) 
def next_customer(customerList):
    # check if customerList is not empty
    if len(customerList)!=0:
        # set nextCustomer to the next element in the list
        nextCustomer=customerList[0]
    # return the next customer
    return nextCustomer

In [ ]:
# for test
next_customer([5, 3, 6])

In [ ]:
# function: finding the nearest depot (nearest_depot included incase lorry fuel refill is required)
def nearest_depot(currentState, depotList):
    # intialise dist dict
    dist = {}

    # loop through depotList
    for i in depotList:
        # update dist with depot and their relative distance values
        dist.update({i:euclidean[currentState, i]})
    
    # intialise temp list
    temp = []

    # loop through dist.keys()
    for i in dist.keys():
        # add values of each dist.keys() to temp 
        temp.append(dist[i])

    # get lowest depot weight
    _shortestDist = min(temp)

    # get the relative node index of the closest depot
    _nearestDepotIndex = np.where(euclidean[currentState]==_shortestDist)

    # return next depot index with relative weight
    return int(_nearestDepotIndex[0]), _shortestDist

In [ ]:
# test near_depot()
nearest_depot(1, depot_locations)

genetic algorithm for exploring solution set

In [ ]:
# imports 
import random

# function to generate a random path order (consider as genome) for each depot # returns a dict 
def randomPathArrangement(cluster):
    randomPathArr = {}
    customerAllocation = cluster
    for depot in depot_locations:
        # append depot as key and list of randomly arranged customer nodes to randomPathArr
        randomPathArr.update({depot:random.sample(customerAllocation[depot], len(customerAllocation[depot]))})
    return randomPathArr

In [ ]:
# function to generate multiple initial solutions (population) # returns genomes (individual solution) in a list
def population_init(cluster, size):
    population = []
    for i in range(0, size):
        population.append(randomPathArrangement(cluster))
    return population

In [ ]:
# return populationFitness at each index...
def fitness(population, edges):

    # to store each weight per traversal made between two nodes (when calculating traversal distance per depot)
    genomeWeight = []

    # for a possible solution in the set of solutions (genome in population)
    for genome in population:
        # initialise genomeFitness dict to store distances at each depot per genome 
        genomeFitness = {}
        for depot in genome.keys():
        #for depot in depot_locations:
            genomeFitness.update({depot:[]})
            nodes = genome[depot]
            # loop for length of nodes
            for idx in range(len(nodes)-1):
                # set nodeA to current loop index
                nodeA = nodes[idx]
                # set nodeB to next loop index
                nodeB = nodes[idx+1]
                # get sum of each traversal distance occurence between two nodes | routing() enables this 
                distance = sum(routing(nodeA, nodeB, edges)[2])
                genomeFitness[depot].append(distance)
        # update popFitness list with each genome and relative fitness values
        genomeWeight.append(genomeFitness)

    genomeWeights = []
    for genome in genomeWeight:
        # initialise popWeight list to store total weight at each depot for genome
        depotWeight = []
        # loop through each depot (chromosome per genome)
        for depot in genome:
            depotWeight.append({depot:sum(genome[depot])})
        genomeWeights.append(depotWeight)
    
    # to store the weight per index (weight per genome) [each solution indicated by index i.e., 0, 1, 2 etc.]
    weightIndex = {}
    for idx, genome in enumerate(genomeWeights):
        tempIndexStore = []
        for depot in genome:
            for i in depot.values():
                tempIndexStore.append(i)
        weightIndex.update({idx:sum(tempIndexStore)})
    
    return population, weightIndex

In [ ]:
# selection() uses weighted selection probability for returning 2 cluster arrangement (2 genomes)
def selection(population, popWeights):
    # selectionPair defines a list containing # of genomes selected through concept of roulette-wheel (in this case k=2). 
    selectionPair = random.choices(population=population, weights=popWeights, k=2)
    # return the selectionPair (in form list)
    return selectionPair 

In [ ]:
# crossover() takes two routes (genomes) and performs crossover operation to produce new routes (offspring)
def crossover(selectionPair):
    # initialise routeA (i.e., genome 1)
    routeA = selectionPair[0]
    # initialise routeB (i.e., genome 2)
    routeB = selectionPair[1]
    
    # check if both routes (both genomes) have same number of keys (depots)
    if len(routeA.keys()) != len(routeB.keys()):
        # give error warning
        print(f'#---crossover error---# route depots are not of same length')
    # if route keys are of same length then proceed with crossover
    else:
        # set random crossover point nPoint
        nPoint = random.randint(1, len(routeA.keys())-1)
        # get routeA.values() up to nPoint 
        extractRouteA1 = list(routeA.values())[nPoint:]
        # get routeB.values() up to nPoint
        extractRouteB1 = list(routeB.values())[nPoint:]
        # get routeA.values() beyond nPoint
        extractRouteA2 = list(routeA.values())[:nPoint]
        # get routeB.values() beyond nPoint
        extractRouteB2 = list(routeB.values())[:nPoint]

        # set offsprings as corresponding concantenation of each route extracts
        offspringA = extractRouteB2+extractRouteA1
        offspringB = extractRouteA2+extractRouteB1

        # intialise offspring dicts
        newRouteA = {}
        newRouteB = {}

        # loop through each depot nodes
        for i, depot in enumerate(depot_locations):
            # allocate new routes to relative depots
            newRouteA.update({depot:offspringA[i]})
            newRouteB.update({depot:offspringB[i]})

    # return the new offsprings
    return newRouteA, newRouteB 

In [ ]:
# mutation() selects a random depot in a route and re-arranges the order of nodes at this depot 
def mutation(crossedPair, mutationRate):
    # get genome routeA
    routeA = crossedPair[0]
    # get genome routeB
    routeB = crossedPair[1]
    # get MUTATION_RATE
    MUTATION_RATE = mutationRate
    # randomly generate a decisive float
    decision = random.uniform(0, 1)
    
    # check if mutation is active
    if decision < MUTATION_RATE:
        # select random mutation point
        mutationPoint = random.choice(depot_locations)
        # extract routes of the mutation point (extract nodes from depot)
        toMutateRouteA = routeA[mutationPoint]
        toMutateRouteB = routeB[mutationPoint]
        # mutate routes (re-arrange the node order) 
        mutateRouteA = random.sample(toMutateRouteA, k=len(toMutateRouteA))
        mutateRouteB = random.sample(toMutateRouteB, k=len(toMutateRouteB))
        # set the mutated routes as offsprings 
        routeA[mutationPoint] = mutateRouteA
        routeB[mutationPoint] = mutateRouteB
        # return mutated routes
        return routeA, routeB

    # other wise return non-mutated routes
    return routeA, routeB

In [ ]:
# helper function to elect two routes (with poor fitness/largest ovr. distance) to be replaced with the new off spring
def replacePopulation(population, popWeight, newPair):

    # sort the population index in descending order via weight (index of longest distance first)
    sortPopWeight = sorted(popWeight.items(), key=lambda idx: idx[1], reverse=True)

    # initialise popIdxToReplace list to store indexes of the less fit genomes to be replaced 
    popIdxToReplace = []
    for sortedWeightIdx in sortPopWeight:
        # check for the number of genomes to be replaced 
        if len(popIdxToReplace) != int(len(newPair)):
            popIdxToReplace.append(sortedWeightIdx[0])
    
    # verify replacement constraints
    if len(popIdxToReplace)==len(newPair):
        # enumerate popIdxToReplace, so that each population index can be replaced with the corresponding index of newPair
        for i, popIdx in enumerate(popIdxToReplace):
            # replace population index with corresponding index of the offspring routes
            population[popIdx] = newPair[i]
    
    # return the updated population
    return population

In [ ]:
# helper function to retain fitter population of routes in the next iterations (preserves 50% from previous iteration)
def retainBestRoutes(currentPopulation, popWeight):

    # sort the population index in ascending order (shortest ovr. distance to largest over. distance)
    sortPopWeight = sorted(popWeight.items(), key=lambda idx: idx[1])
    
    # initilaise popRetainIdx list to store indexes of population to retain
    popIdxToRetain = []
    for sortedWeightIdx in sortPopWeight:
        # check loop for 50% of the popluation 
        if len(popIdxToRetain) != int(len(currentPopulation)/2):
            popIdxToRetain.append(sortedWeightIdx[0])
    
    # initialise popRetain list to store list of population
    popRetain = []
    # match popIdx with currentPopulation to append to popRetain list
    for popIdx in popIdxToRetain:
        popRetain.append(currentPopulation[popIdx])
    
    # return list of population to retain for next iteration
    return popRetain 

In [ ]:
# helper function to elicit best route per iteration 
def getBestRoute(currentPopulation, popWeight):
    # sort weights in ascending order and extract the index of the shortest overall route
    sortPopWeight = sorted(popWeight.items(), key=lambda idx: idx[1])
    bestRouteIdx, routeDistance = sortPopWeight[0][0], sortPopWeight[0][1]
    bestRoute = currentPopulation[bestRouteIdx]
    return bestRoute, routeDistance

In [ ]:
# run genetic algorithm to search for the most optimal route; termination criteria = iterations [keep 'size' EVEN int]
def runGeneticAlgorithm(cluster, size, iterationLimit, edges):
    
    # generate initial population 
    population = population_init(cluster, size)
    # initialise list to store upcoming population
    nextPopulation = []
    # set iteration count to 0
    iteration = 0

    # while iteration limit not exceeded
    while iteration < iterationLimit:
        print(f'\n\niteration: {iteration}')
        # check for retained population 
        if nextPopulation:
            population = nextPopulation
            # if population list has enough set of routes 
            if int(len(population)) != int(len(population)*2):
                # generate half the number of initial size to append to the existing list of population NB! key[0] to unpack list
                population.append(population_init(cluster, int(size/2))[0])
            print(f'population: {population}')
        
        # get initial fitness of each routes and assign the value as weights per genome 
        popWeights = fitness(population, edges)[1]
        # select two parent genomes (two routes)
        selectionPair = selection(population, popWeights)
        # perform the crossover operation 
        crossedPair = crossover(selectionPair)
        # mutation stage and not 'mutated' because mutation occurs at random rates
        mutationStagePair = mutation(crossedPair, mutationRate=0.4)
        # replace the population with new offsprings at the corresponding indexes
        population = replacePopulation(population, popWeights, mutationStagePair)
        # get the new fitness per route in the population index
        popWeights = fitness(population, edges)[1]
        # filter populations to preserve '50%' of the fittest routes in the current population for next iteration
        nextPopulation = retainBestRoutes(population, popWeights)
        print(f'nextPopulation: {nextPopulation}')

        # iteration increment
        iteration += 1
    
    # reset popWeights after loop to eval. new pop
    popWeights = fitness(population, edges)[1]

    # gets the shortest route found and the route distance
    bestRoute = getBestRoute(population, popWeights)

    # return the bestRoute along with route distance (tuple)
    return bestRoute
    

In [ ]:
# store the results of runGeneticAlgorithm() in getRoute
getRoute = runGeneticAlgorithm(cluster=cluster, size=4, iterationLimit=4, edges=edges)

In [ ]:
# get best route found by the G.A.
bestRoute = getRoute[0]
# get the distance of the best route
routeDistance = getRoute[1]
# display route and distance
print(f'{bestRoute}\n\n{routeDistance}')

In [ ]:
# initailise routes dict 
routes = {}

# loop through lorry_df index
for i in lorry_df.index:
    # for each lorry, initialise journey by appending start (depot) and capacity (self)
    routes.update({lorry_df['lorry_id'][i]:[[int(lorry_df.depot[i]), lorry_df.capacity[i]]]})

In [ ]:
# display routes
print(routes)

In [ ]:
# initialise routeTotalCost list to store route costs
routeTotalCost = []
# initialise totalGasDelivered list to store the amount of gas delivered
totalGasSupplied = []

In [ ]:
#tempRoutes = {'124-0': [[124, 8]], '124-1': [[124, 8]], '124-2': [[124, 16]], '124-3': [[124, 25]], '127-0': [[127, 8]], '127-1': [[127, 16]], '127-2': [[127, 25]], '167-0': [[167, 8]], '167-1': [[167, 16]], '167-2': [[167, 16]], '167-3': [[167, 25]], '523-0': [[523, 8]], '523-1': [[523, 16]], '523-2': [[523, 25]]}
#tempCluster = {124: [418, 170, 627, 327, 129, 633, 492, 369, 626, 87, 542, 196, 263, 561, 264, 252, 336, 476, 461, 140, 43, 297, 254, 266, 209, 566, 210, 15, 71, 470, 130, 288, 606, 149, 572, 278, 616, 37, 321, 10, 281, 503], 127: [471, 534, 562, 26, 269, 357, 416, 583, 178, 126, 609, 302, 215, 279, 96, 198, 586, 515, 387, 77, 532, 151], 167: [514, 316, 144, 469, 190, 25, 548, 86, 466, 155, 454, 348, 390, 313, 431, 282, 398, 468, 478, 381, 356, 569, 64, 243, 489, 166, 75, 72, 595, 193, 513, 615, 580, 485, 393, 319, 621, 22, 141, 20], 523: [497, 564, 235, 508, 332, 152, 276, 338, 630, 490, 176, 7, 401, 367, 456, 588, 253]}

In [ ]:
def route(lorryId, allocatedCustomers):
    # get currentId of lorry
    currentId = lorryId
    # get base depot of lorry
    operatingDepot = currentId.split('-')[0]
    # get currentCapacity of lorry
    currentCapacity = float(lorry_df[lorry_df['lorry_id']==currentId].capacity)
    # get currentCpm of lorry
    currentCpm = float(lorry_df[lorry_df['lorry_id']==currentId].cpm)
    # get currentCptm of lorry
    currentCptm = float(lorry_df[lorry_df['lorry_id']==currentId].cptm)
    # get currentState of lorry
    currentState = int(lorry_df[lorry_df['lorry_id']==currentId].depot)
    # get current depot of lorry
    currentDepot = int(lorry_df[lorry_df['lorry_id']==currentId].depot)
    # set next customer capacity at 0
    nextCustomerCapacity = 0

    # iterate while the lorry capacity is >= the remaining customer capacity and while their are remaining allocated customers
    while (currentCapacity >= nextCustomerCapacity) and (len(allocatedCustomers)>0):
        # set next element in allocatedCustomers as the nextCustomer
        nextCustomer = next_customer(allocatedCustomers)

        # check if next customer tank capacity is more than 80%
        if ((location_df[location_df['id']==nextCustomer]['capacity'].iloc[0])-float(location_df[location_df['id']==nextCustomer]['required'].iloc[0])) < ((location_df[location_df['id']==nextCustomer]['capacity'].iloc[0])):
        # if float((location_df[location_df['id']==nextCustomer]['capacity'].iloc[0])-(location_df[location_df['id']==nextCustomer]['required'].iloc[0]))<float((location_df[location_df['id']==nextCustomer]['capacity'].iloc[0])*float(0.8)):
            # prepare 80% fuel amount to fill up to 80% of relative customer tank capacity
            fillAmount = float((location_df[location_df['id']==nextCustomer]['capacity'].iloc[0])*float(0.8))-float(location_df[location_df['id']==nextCustomer]['required'].iloc[0])
            # add the amount of gas supplied to totalGasDelivered
            totalGasSupplied.append(fillAmount)
            routes[currentId].append([int(nextCustomer), -fillAmount])
            #tempRoutes[currentId].append([int(nextCustomer), -fillAmount])
            # get the cost of traversing between each node links when travelling to nextCustomer node (round to 2 d.p.)
            routeCost = round(sum(routing(currentState, nextCustomer, edges)[2])*(currentCpm+(currentCapacity*currentCpm)), 2)
            # append routeCost to routeTotalCost list for post-route cost eval.
            routeTotalCost.append(routeCost)
            # update the current lorry capacity after gas deposit
            currentCapacity = currentCapacity-(location_df[location_df['id']==nextCustomer]['required'].iloc[0])
            # update the allocatedCustomer's list by removing the served customers
            allocatedCustomers.remove(nextCustomer)
            # update the lorry's currentState 
            currentState = nextCustomer

        # check if customer tank space is less than 80% of their capacity (then they do not require filling)
        elif ((location_df[location_df['id']==nextCustomer]['capacity'].iloc[0])-float(location_df[location_df['id']==nextCustomer]['required'].iloc[0])) > ((location_df[location_df['id']==nextCustomer]['capacity'].iloc[0])):
        # elif float((location_df[location_df['id']==nextCustomer]['capacity'].iloc[0])-(location_df[location_df['id']==nextCustomer]['required'].iloc[0]))>float((location_df[location_df['id']==nextCustomer]['capacity'].iloc[0])*float(0.8)):
            # -0 deposit since customer doesn't require filling
            fillAmount = float(0)
            routes[currentId].append([int(nextCustomer), -fillAmount])
            #tempRoutes[currentId].append([int(nextCustomer), -fillAmount])
            # get the cost of traversing between each node links when travelling to nextCustomer node (round to 2 d.p.)
            routeCost = round(sum(routing(currentState, nextCustomer, edges)[2])*(currentCpm+(currentCapacity*currentCpm)), 2)
            # append routeCost to routeTotalCost list for post-route cost eval.
            routeTotalCost.append(routeCost)
            # update the current lorry capacity after gas deposit
            currentCapacity = currentCapacity-(location_df[location_df['id']==nextCustomer]['required'].iloc[0])
            # if tank capacity is above 80% then remove customer as they do not require filling
            allocatedCustomers.remove(nextCustomer)
            # update the lorry's currentState 
            currentState = nextCustomer

        # check for remaining allocated customers
        if len(allocatedCustomers) != 0:
            # set nextCustomerCapacity to the nearest customer's capacity for consecutive iterations
            nextCustomerCapacity = location_df[location_df['id']==(next_customer(allocatedCustomers))]['required'].iloc[0]

    # if lorry's currentCapacity < nextCustomerCapacity
    if currentCapacity < nextCustomerCapacity:
        # update the cluster's customer list with the current set of allocatedCustomers
        cluster[currentDepot]=allocatedCustomers
        # print current lorry's end of route
        print(f'{currentId} finished operating at {currentState}')
    
    # check if any allocated customers remaining
    elif len(allocatedCustomers)==0:
        # print current lorry's  end of route
        print(f'{currentId} finished operating at {currentState}')

In [ ]:
# loop through each depot in depot_locations
for i in depot_locations:
    # initialise relative dataframe for lorries at each depot
    df = lorry_df[lorry_df['depot']==f'{i}']
    # set count to 0
    count = 0 

    # loop through each index per in-loop defined df (of depot-relative lorries)
    for j in df.index:
        # print start of lorry's route
        print(f'{df.loc[j].lorry_id} started operating')
        # call route() for each lorry in the relative depot
        route(df.loc[j].lorry_id, bestRoute[i])
        # count increment
        count += 1
        
        # check if depot's allocated customers remaining
        if len(bestRoute[i])!=0 and count==len(df.index): 
            # while depot has customers remaining
            while len(bestRoute[i]!=0):
                # set currentId to relative lorry's ID
                currentId = lorry_df.loc[j]['lorry_id']
                # get the last state of the relative lorry from routes dict
                currentState = routes[currentId][-1][0]
                # find the nearest depot from the lorry's current state
                nearDepot = nearest_depot(currentState)
                # print refill statement
                print(f'{currentId} is refilling tank at {nearDepot[0]}')
                # get the cost of traversing between each node links when travelling to nearDepot node (round to 2 d.p.)
                routeCost = round(sum(routing(currentState, nearDepot[0], edges)[2])*((lorry_df.loc[j]['cpm'])+(lorry_df.loc[j]['capacity']*lorry_df.loc[j]['cptm'])), 2)
                # append routeCost to routeTotalCost list for post-route cost eval.
                routeTotalCost.append(routeCost)
                # update routes dict with new state and refill amount
                routes[currentId].append([int(nearDepot[0]),int(lorry_df.loc[j]['capacity'])])
                #tempRoutes[currentId].append([int(nearDepot[0]),int(lorry_df.loc[j]['capacity'])])
                # update currentState to nearDepot
                currentState = nearDepot[0]
                # call route() for the relative lorry with the relative allocated customers
                route(currentId, bestRoute[i])
        # check if depot has customers remaining
        elif len(bestRoute[i])==0:
            # end loop if no customers remaining
            break

In [ ]:
routes

In [ ]:
costEfficiency = sum(totalGasSupplied)/sum(routeTotalCost)
print(f'TOTAL GAS SUPPLIED: {sum(totalGasSupplied)}\nPATH COST: {round(sum(routeTotalCost), 2)}\nCOST EFFICIENCY: {costEfficiency}')

In [ ]:
# initialise results list
results = []
# store the lorry_id and relative traversal made in routes to results
for i in routes.keys():
    results.append({'lorry_id':i, 'traversal':routes[i]})

In [ ]:
# an encoder class which helps to make the results list json serializable 
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
         if isinstance(obj, np.integer):
            return int(obj)
         elif isinstance(obj, np.floating):
            return float(obj)
         elif isinstance(obj, np.ndarray):
            return obj.tolist()
         else:
            return super(NpEncoder, self).default(obj)

In [ ]:
# open new file 'task1_solution.json' as file for outputting results in json format
with open('task2_solution.json', 'w') as file:
    # use json.dump() to convert type(results) to json and stream to filepath 
    json.dump(results, file, cls=NpEncoder)